# Introduction to DataFrames, Visualization, and Statistics in Python

Run this cell to `import` the various packages that we will be using today:

In [ ]:
import numpy as np  # numerical python lib
import pandas as pd  # tabular data analysis
import matplotlib.pyplot as plt  # plotting
import seaborn as sns  # nicer plotting
sns.set_style('whitegrid')  # set plot style
%matplotlib inline
import statsmodels.formula.api as smf  # basic statistical modeling

## Table of Contents

1 - [Introduction to `pandas` and `DataFrame`s](#pandas)

2 - [Manipulating `DataFrame`s: joining, pivoting, grouping](#manipulating)

3 - [Visualization](#visualization)

4 - [Statistics and Correlation](#correlation)


## 1. Introduction to `pandas` and `DataFrame`s <a id='pandas'></a>

Many of you may be familiar with the spreadsheet software Excel, or even `DataFrame`s in the R programming language. For tabular data, the most commonly used data analysis package in Python is called `pandas`. What we call tables or spreadsheets, `pandas` calls `DataFrame`s. If you've worked in the R programming language before, you'll have a head start with `pandas` because much of the syntax used for R dataframes carries over!

You will often see `DataFrame`s abbreviated as `df` in code examples. Run the cell below to see a (very simple) example of a `DataFrame`.

In [ ]:
df = pd.read_csv('./Data/01/ds_courses.csv')
df

We know this is a `DataFrame` by using the `type` function:

In [ ]:
type(df)

Look at the structure of the `DataFrame`. Each column represents a different attribute of a row. For example, the class title corresponding to the course "Data 8" is "Foundations of Data Science". The number on the left of the `DataFrame` corresponds to the "index" of the row. For example, entry 4 (which is actually the 5th row from the top, because Python uses 0-based indexing!) corresponds to the row with L&S 88-2. Now that you know what a `DataFrame` is, let's move on to making our own from scratch!

### Making `DataFrame`s
There are two basic ways of creating `DataFrame`s. The first is to make one by typing the data into Python objects manually. For example, we can make `numpy` arrays that correspond to different attributes. Note that the order of attributes in each array matters, and each array must be the same size. Run the cell below to see an example of what this means. When we make the `DataFrame` itself, the input to the function `pd.DataFrame()` is a Python dictionary with the column title and the array with the data in it.

In [ ]:
# one column
journal_titles = np.array(['Nature Reviews Molecular Cell Biology',
                           'Nature Methods',
                           'Nature Cell Biology',
                           'Cell Stem Cell',
                           'Molecular Cell',
                           'Cancer Cell',
                           'Cell Metabolism',
                           'Genome Biology',
                           'Trends in Cell Biology',
                           'Annual Review of Biophysics'])

# another column
journal_impacts = np.array([29.656, 
                            19.544, 
                            14.110, 
                            13.515, 
                            13.295, 
                            13.169, 
                            11.209, 
                            10.484, 
                            10.113, 
                            9.801])

# make df
journal_df = pd.DataFrame({'Title': journal_titles,
                           'Impact Factor': journal_impacts}) 

journal_df

As you can see, this made us a `DataFrame`! However, this will obviously get very tedious for very large datasets. For large datasets we can read in existing data files. You can import a `.csv` file using the function `pd.read_csv()`. Run the following cell to see an example of this in action.

In [ ]:
family_heights = pd.read_csv('Data/01/galtonfamilies.csv')
family_heights

We see we've read in the famous Galton family dataset with heights for members of different families.

If you scroll to the bottom, you can see that this `DataFrame` has 934 rows and 9 columns in it! Typing this out by hand would clearly take a very very long time. If you have an extra large dataset (or don't know how big it is!) we can use the `.head()` method to give us the first 5 rows so we don't freeze our notebook:

In [ ]:
family_heights.head()

### Working with `DataFrame`s
We can use the `.columns` attribute to get a `list` of columns:

In [ ]:
family_heights.columns

If we use the `len` function on this, we'll see the number of columns

In [ ]:
len(family_heights.columns)

We can use `len` on the `DataFrame` object itself to get the number of rows:

In [ ]:
len(family_heights)

We can use `df.shape` to give us both. The 0th entry in the output of `df.shape` is the number of rows in the `DataFrame`, and the 1st entry in the output of `df.shape` is the number of columns in the `DataFrame`.

In [ ]:
family_heights.shape

In [ ]:
print('Number of rows: ' + str(family_heights.shape[0]))
print('Number of columns: ' + str(family_heights.shape[1]))

#### Subsetting

Sometimes we also might not care about some of the columns. Keeping them around can often be detrimental to the performance of Python, especially on very large datasets. Luckily, we can either choose the columns we want using `df[lst]`, where `lst` is a list of columns we want to keep:

In [ ]:
cols_after_index = family_heights[['family', 'father', 'mother', 'gender', 'childheight']]
cols_after_index

or delete the columns we don't want using `df.drop(lst, axis=1)`, where `lst` is a list of columns we don't want to keep, but we'll have to reassign this to a variable. We need the parameter `axis=1` to tell Pandas that we are dropping a column, not a row. The following cell gives us the exact same output as the one above:

In [ ]:
cols_after_drop = family_heights.drop(['id', 'midparentheight', 'children', 'childnum'], axis=1)
cols_after_drop

If we want to **filter** rows, we can simply "index" into our `DataFrame` using a condition. `df[df.column_name == some_value]` will give us all the rows corresponding to entries where the `column_name` property equals `some_value`. For example, let's say we only wanted to look at the heights of sons in the table of heights:

In [ ]:
only_males = family_heights[family_heights.gender == 'male']
only_males

As you can see, this returned to us a table with only the rows that had `'male'` in the `gender` column! Note that this can work with any condition. For example, if we only wanted the rows where the height was greater than a certain value, we can also index into the table with a `>` condition:

In [ ]:
taller_than_70 = family_heights[family_heights.childheight > 70]
taller_than_70

We could calculate the fraction of pairings where the mother is taller than the father by using a combination of subsetting and the `len` function:   

In [ ]:
taller_mother = len(family_heights[family_heights.mother > family_heights.father]) / len(family_heights)
taller_mother

---

If you want to index a specific row, you can use the `.iloc` method. With `.iloc` you give the row index (or slice) followed by the column index (or slice). You *need* the comma separting row and column indices.

Here is the fourth row:

In [ ]:
family_heights.iloc[3, ]

The second row and 5th column:

In [ ]:
family_heights.iloc[1, 4]

Rows 2-10 and columns 3-4

In [ ]:
family_heights.iloc[1:10, 2:4]

---

## 2. Manipulating `DataFrame`s: Grouping, Pivoting, and Joining <a id='manipulating'></a>

What if we wanted to seperately study the heights of the sons and the daughters of each family?

Let's create a separate `DataFrame` containing the height information for just the parents. First, select the columns containing the family code, height of father, height of mother, and average height of parents.

In [ ]:
parent_heights = family_heights[['family','father', 'mother', 'midparentheight']]
parent_heights

You can see that each entry of the data structure still corresponds to an individual child. We want to reduce the number of entries so that there is only one row for every family. We can do this by **grouping** the rows of the data structure by family.

***Note***: There is only one set of parents per family, so taking the `mean` of parents for each family is just collapsing the rows. 

In [ ]:
parent_heights = parent_heights.groupby('family').mean()
parent_heights

Now that we have a `DataFrame` of `parent_heights`, let's make a `DataFrame` for the heights of the children! From the original `DataFrame`, create a new `DataFrame` called `children_heights` which contains the family code, gender, and height of each child.

In [ ]:
children_heights = family_heights[['family', 'gender', 'childheight']]
children_heights

Let's now look at the average daughter and son heights for each family. We'll need the `.pivot` method to flip the rows and columns. In this case, we can use `.pivot` to find the average heights of male and female children in each of the families in the data set.

In [ ]:
average_child_height_by_gender = pd.pivot_table(children_heights, values = 'childheight', index = ['family'], columns = ['gender'])
average_child_height_by_gender

Now that we have tables for both parent and child height, we want to somehow combine the two `DataFrame`s so we can make comparisons between parent and child height. We can use **join**, which merges two `DataFrame`s based on a shared attribute. In this case, `parent_heights` and `average_child_height_by_gender` both have a `'family'` index. Joining the two tables would match columns with each other in accordance to their `'family'`.

In [ ]:
family_average_heights = parent_heights.join(average_child_height_by_gender)
family_average_heights

In the table above you'll see some `NaN` (Not-a-Number) values. This name is a little misleading, in `pandas`, `NaN` is used for any missing value. Any calculations with these values will either error out or run silently by dropping the entries.

There are many ways to impute the values for missing data, which we won't cover now. We will show the `.dropna()` function that will drop out those rows entirely, but this effectively drops any family that doesn't have both a daughter and a son:

In [ ]:
family_average_heights_nona = family_average_heights.dropna()
family_average_heights_nona

---

### Challenge Problems!

**1. Using the original `family_heights` table, create a new table `mothers_over_65` that contains the columns `family`, `father`, and `mother` where all of the mothers heights are over 65. **

In [ ]:
mothers_over_65 = # Replace this line with your code!
mothers_over_65

In [ ]:
### SOLUTION ###
mothers_over_65 = family_heights[['family', 'father', 'mother']]
mothers_over_65 = mothers_over_65[mothers_over_65.mother > 65]
mothers_over_65

**2. Try creating a new table `average_heights_of_parents_of_males` by grouping the `family_heights` table by `male` to find the average heights of the mothers and fathers of all of the sons of a certain height. Only show the columns `male`, `father`, and `mother`.**

In [ ]:
average_heights_of_parents_of_males = # Replace this line with your code!
average_heights_of_parents_of_males

In [ ]:
### SOLUTION ###
average_heights_of_parents_of_males = family_average_heights.groupby('male').mean().drop(['midparentheight', 'female'], axis=1)
average_heights_of_parents_of_males

---

## 3. Visualization: Histograms and Scatter Plots <a id='visualization'></a>

### Histograms
Let's use `family_average_heights` to compare the children's heights to their parents! Start off by finding the difference between the daughters' heights to their mothers' heights. Add these values as a column named `mother_daughter_height_difference` to the `family_average_heights` `DataFrame`.

In [ ]:
mother_daughter_difference = family_average_heights['female'] - family_average_heights['mother']
family_average_heights['mother_daughter_height_difference'] = mother_daughter_difference
family_average_heights

Let's plot `mother_daughter_height_difference` on a histogram to see the distribution of height differences.

In [ ]:
family_average_heights['mother_daughter_height_difference'].plot.hist()

By looking by the histogram, we see that the **mean** of the height difference between mothers and daughters is around 0. This means, that on average, daughters tend to be around the same height as their mothers. However, the wide distribution of height differences makes it difficult for us to conclude a relationship between daughter and mother height.

Let's calculate how much taller (or shorter) each of the fathers are compared to the mothers. Subtracting a column from a column returns a column.

In [ ]:
difference = family_heights["father"] - family_heights["mother"]
difference

We can find the `mean` by calling `pd.DataFrame.mean(*argument)`:

In [ ]:
difference.mean()

We can also visualize each of the individual height differences with a histogram like above:

In [ ]:
difference.plot.hist()

---

### Scatter plots

Let's look at a scatterplot between mother heights and daughter heights now. We can do this by applying the `.scatter()` function

`.scatter()` has two required arguments: `x` and `y`, which refer to the attributes of the `DataFrame` that you are plotting against each other. `x` will correspond to the horizontal axis of the plot, while `y` will correspond to the vertical axis. 

`.scatter()` also takes in optional arguments `s`, `c`, and `ax`. `s` is an integer describing the desired size of the points on the plot. `c` is a string describing the desired color of the points. `axl` can be used to overlay one scatter plot onto another. We will see an example of an overlay plot later in this notebook.

Let's create a scatter plot using `family_average_heights` to compare the relationship between `'mother'` and `'daughter'`. Set the size of the points to `20` and the color to `'blue'`.

In [ ]:
family_average_heights.plot.scatter(x = 'mother', y = 'female', s = 20, c = 'blue')

We see that there is not a clear relationship between the two heights, and are hesitant to draw any conclusions about the relationship between mother and daughter heights. Maybe there is a relationship between father and daughter heights!

Let's check by creating a scatter plot comparing `'father'` and `'female'`. Set the size of the points to be `20`, and the color of the points to be `red`.

In [ ]:
family_average_heights.plot.scatter(x = 'father', y = 'female', s = 20, c = 'red')

The relationship between father and daughter heights isn't any more clear than the relationship between mother and daughter heights was! 

Let's instead look at the relationship between the height of daughters and the average height of their parents. Create a scatter plot that compares `'midparentheight'` and `'female'`. Set the size of the points to be `35` and the color of the points to be `'green'`.

In [ ]:
family_average_heights.plot.scatter(x = 'midparentheight', y = 'female', s = 35, color = 'green')

We can see that the average parent height and daughter height appear to have more of a linear relationship. This makes sense, since inheritance of height is dependent on both parents, not just the mother or father. 

We can further see the difference between the relationship between average parent height and daughter height and mother height and daughter height by overlaying the scatterplots that we had previously created.

We can do this by passing a value through as the optional `ax` argument in our `.scatter()` function. 

Define `axl` to be the scatterplot for mother and daughter heights. Then, create a scatterplot for the average height of parents and daughter height, passing `ax = axl` as an argument in `.scatter()`.

In [ ]:
axl = family_average_heights.plot.scatter(x = 'mother', y = 'female', s = 20, c = 'blue')

family_average_heights.plot.scatter(x = 'midparentheight', y = 'female', s = 35, color = 'green', ax = axl)

### Challenge Problems!

Now try creating scatterplots as the ones above, but for comparisons with the heights of sons.

**1. Create a scatterplot using the heights of mothers and sons. Make the size of the points `15` and the color of the points `gold`. **

In [ ]:
# Replace this line with your code!

In [ ]:
### SOLUTION ###
family_average_heights.plot.scatter(x = 'mother', y = 'male', s = 15, c = 'gold')

**2. Create a scatterplot using the heights of fathers and sons. Make the size of the points `15` and the color of the points `purple`. **

In [ ]:
# Replace this line with your code!

In [ ]:
### SOLUTION ###
family_average_heights.plot.scatter(x = 'father', y = 'male', s = 15, c = 'purple')

**3. Create a scatterplot using the height of sons and the average height of their parents.. Make the size of the points `35` and the color of the points `grey`. **

In [ ]:
# Replace this line with your code!

In [ ]:
### SOLUTION ###
family_average_heights.plot.scatter(x = 'midparentheight', y = 'male', s = 25, color = 'grey')

**4. Create a scatterplot overlaying the plot of son and father heights and the plot of average parent heights and son heights.**

In [ ]:
# Replace this line with your code!

In [ ]:
### SOLUTION ###
axl = family_average_heights.plot.scatter(x = 'father', y = 'male', s = 15, c = 'purple')

family_average_heights.plot.scatter(x = 'midparentheight', y = 'male', s = 25, color = 'grey', ax = axl)

---

## 4. Correlation and Regression in Python <a id='correlation'></a>

### Correlation

We saw above that there's a positive correlation between midparent heights and female child heights according to our scatter plot. Let's see this again:

In [ ]:
family_average_heights.plot.scatter(x = 'midparentheight', y = 'female', s = 35, color = 'green')

`pandas` has the handy `.corr()` method to quickly calculate a correlation matrix:

In [ ]:
family_average_heights.corr()

This function returns a matrix. We have a diagonal of 1s because, of course, `father` is correlated perfectly with `father`, and `mother` is correlated perfectly with `mother`, etc. Our coefficient here on `midparentheight` and `female` is 0.594085, ***implying positive correlation***.

With just one line of code, we were able to draw quantitative conclusions from our data! Let's perform some more statistical analysis.

---

### Regression

For our statistical analysis, we will be using a Python library called `statsmodels`. It provides functions and classes for estimation, analysis and creation of numerous different statistical models. Using this model, we can also conduct important statistical tests.

In the following cell, we will be creating our linear regression model and printing a summary of it. The library is imported as `smf`, and we use the `ols` function (ordinary least squares) to develop the model. We then fit it to the data using the `fit` function. We input a formula in the form of 'y-axis ~ x-axis' and pass in our desired Dataframe. The `formula` should mirror our regression:

$$FEMALE_i= \alpha + \beta MIDPARENTHEIGHT + \epsilon_i$$

In [ ]:
model = smf.ols(formula='female ~ midparentheight', data=family_average_heights).fit()
model.summary()

We can get back the parameters from the `params` attribute:

In [ ]:
p = model.params
p

Let's plot our best fit line on our scatter plot! First, we need to create our x-axis, which we'll do by getting a range from the `min` to the `max` of `midparentheight`:

In [ ]:
minX = min(family_average_heights['midparentheight'])
maxX = max(family_average_heights['midparentheight'])
x = np.arange(minX, maxX)
x

In the cell below, we'll first `scatter` the two variables as we have before. Then we'll `plot` a line with our x-axis `x` points, our y-axis points from the regression, with a `color` of red. Then we'll add in our `xlabel` and `ylabel`:

In [ ]:
plt.scatter(family_average_heights['midparentheight'], family_average_heights['female'])  # creating the old scatter plot
plt.plot(x, p['Intercept'] + p['midparentheight']*x, color = 'r')  # plotting 'line' variable on the x-range
plt.xlabel("midparentheight")
plt.ylabel("female")

Notice that while the line seems to go through the average of the data, there are many points that are significantly off of the line. This is in accordance with our $R^2$ value.

### Challenge Problem

**1. Create a linear regression model for a different set of variables from the DataFrame. Hint: Follow the form of the code from the previous cells closely! **

In [ ]:
# YOUR CODE HERE